<a href="https://colab.research.google.com/github/prabhatkumar13/mHAR-a-novel-convolutional-recurrent-model-for-recognizing-motion-based-human-activity/blob/main/Source_Code_for_mHAR_a_novel_convolutional_recurrent_model_for_recognizing_motion_based_human_activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#YCS-2023_WISDM_mHAR_Model_Source_Code
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

file = open('/content/drive/MyDrive/WISDM/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt')
lines = file.readlines()

processedList = []

for i, line in enumerate(lines):
    try:
        line = line.split(',')
        last = line[5].split(';')[0]
        last = last.strip()
        if last == '':
            break;
        temp = [line[0], line[1], line[2], line[3], line[4], last]
        processedList.append(temp)
    except:
        print('Error at line number: ', i)


columns = ['user', 'activity', 'time', 'x', 'y', 'z']

data = pd.DataFrame(data = processedList, columns = columns)
data.head()

data.shape

data.isnull().sum()

data['x'] = data['x'].astype('float')
data['y'] = data['y'].astype('float')
data['z'] = data['z'].astype('float')

activities = data['activity'].value_counts().index
activities

data['activity'].value_counts()

# Normalize features for training data set (values between 0 and 1)
# Surpress warning for next 3 operation
pd.options.mode.chained_assignment = None  # default='warn'
data['x'] = data['x'].astype(float) / data['x'].astype(float).max()
data['y'] = data['y'].astype(float) / data['y'].astype(float).max()
data['z'] = data['z'].astype(float) / data['z'].astype(float).max()
# Round numbers
data = data.round({'x': 4, 'y': 4, 'z': 4})

static = data[data['activity'].isin (['Sitting', 'Standing'])]
static

static['Encode'] = "Static"

static

gradual = data[data['activity'].isin (['Upstairs', 'Downstairs'])]
gradual

gradual['Encode'] = "Gradual"
gradual

dynamic = data[data['activity'].isin (['Walking', 'Jogging'])]
dynamic

dynamic['Encode'] = "Dynamic"
dynamic

frames = [static, gradual, dynamic
          ]
dataCol = pd.concat(frames)
print(dataCol)
dataCol['Encode'].value_counts()

dataCol

from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
dataCol['label'] = label.fit_transform(dataCol['Encode'])
dataCol.head()

dataCol['label'].value_counts()

dataCol = dataCol.drop(['user', 'activity', 'time', 'Encode'], axis = 1).copy()
dataCol.head()

label.classes_

Fs = 50
frame_size = Fs*4 # 200
hop_size = Fs*2 # 200

import math as m
def get_frames(data, frame_size, hop_size):

  N_FEATURES = 3
  frames = []
  labels = []
  for i in range(0, len(dataCol) - frame_size, hop_size):
    x = data['x'].values[i: i + frame_size]
    y = data['y'].values[i: i + frame_size]
    z = data['z'].values[i: i + frame_size]
        # Retrieve the most often used label in this segment
    label = stats.mode(dataCol['label'][i: i + frame_size])[0][0]
    frames.append([x, y, z])
    labels.append(label)
  # Bring the segments into a better shape
  frames = np.asarray(frames, dtype= np.float32).reshape(-1, frame_size, N_FEATURES)
  labels = np.asarray(labels)
  return frames, labels

from scipy import stats
X,y = get_frames(dataCol, frame_size, hop_size)

X.shape
y.shape

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0, stratify = y)

print(x_train.shape)
print(x_test.shape)

print(y_train.shape)
print(y_test.shape)

import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from keras.layers import Conv1D,MaxPooling1D,Flatten

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, Dropout, MaxPooling1D, Flatten, LSTM,BatchNormalization, SimpleRNN

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu',input_shape=x_train[0].shape))
model.add(LSTM(30))
model.add((Flatten()))
model.add(Dense(3, activation="softmax"))
model.summary()

from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=0.0001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy',f1_m,precision_m,recall_m])

from time import perf_counter
print("Time Counting Started :")
t1_start = perf_counter()

score = model.fit(x_train,y_train, epochs=30, validation_data= (x_test, y_test),batch_size=64, verbose=1)

t1_stop = perf_counter()
print("Elapsed time:", t1_stop, t1_start)
print("Elapsed time during the whole program in seconds:",
                                        t1_stop-t1_start)

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test)

predy = np.argmax(model.predict(x_test), axis=1)

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report

cm=confusion_matrix(y_test,predy)
print(cm)
print(accuracy_score(y_test,predy))
print("classification_report of model1:")

mat = confusion_matrix(y_test,predy)

print('Precision: %.3f' % precision_score(y_test, predy, average='micro'))
target_names = ['class 0', 'class 1', 'class 2']

print(classification_report(y_test,predy, target_names=target_names))

plot_confusion_matrix(conf_mat=mat, show_normed=True)

plot_confusion_matrix(conf_mat=mat, show_normed=True, show_absolute=False, colorbar=True)

import matplotlib.pyplot as plt1

# Plot training & validation accuracy values
plt.figure(figsize=(4,4))
plt.plot(score.history['accuracy'],label='Train accuracy')
plt.plot(score.history['val_accuracy'],label = 'Validation accuracy')
plt.xlabel('epoch no')
plt.ylabel('acc')
plt.legend()
plt.show()

plt.figure(figsize=(4,4))
plt.plot(score.history['loss'],label='Train loss')
plt.plot(score.history['val_loss'],label = 'Validation loss')
plt.xlabel('epoch no')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
#YCS-2023_WISDM_RNN_Model_Source_Code
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

file = open('/content/drive/MyDrive/WISDM/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt')
lines = file.readlines()

processedList = []

for i, line in enumerate(lines):
    try:
        line = line.split(',')
        last = line[5].split(';')[0]
        last = last.strip()
        if last == '':
            break;
        temp = [line[0], line[1], line[2], line[3], line[4], last]
        processedList.append(temp)
    except:
        print('Error at line number: ', i)


columns = ['user', 'activity', 'time', 'x', 'y', 'z']

data = pd.DataFrame(data = processedList, columns = columns)
data.head()

data.shape

data.isnull().sum()

data['x'] = data['x'].astype('float')
data['y'] = data['y'].astype('float')
data['z'] = data['z'].astype('float')

activities = data['activity'].value_counts().index
activities

data['activity'].value_counts()

# Normalize features for training data set (values between 0 and 1)
# Surpress warning for next 3 operation
pd.options.mode.chained_assignment = None  # default='warn'
data['x'] = data['x'].astype(float) / data['x'].astype(float).max()
data['y'] = data['y'].astype(float) / data['y'].astype(float).max()
data['z'] = data['z'].astype(float) / data['z'].astype(float).max()
# Round numbers
data = data.round({'x': 4, 'y': 4, 'z': 4})

static = data[data['activity'].isin (['Sitting', 'Standing'])]
static

static['Encode'] = "Static"

static

gradual = data[data['activity'].isin (['Upstairs', 'Downstairs'])]
gradual

gradual['Encode'] = "Gradual"
gradual

dynamic = data[data['activity'].isin (['Walking', 'Jogging'])]
dynamic

dynamic['Encode'] = "Dynamic"
dynamic

frames = [static, gradual, dynamic
          ]
dataCol = pd.concat(frames)
print(dataCol)
dataCol['Encode'].value_counts()

dataCol

from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
dataCol['label'] = label.fit_transform(dataCol['Encode'])
dataCol.head()

dataCol['label'].value_counts()

dataCol = dataCol.drop(['user', 'activity', 'time', 'Encode'], axis = 1).copy()
dataCol.head()

label.classes_

Fs = 50
frame_size = Fs*4 # 200
hop_size = Fs*2 # 200

import math as m
def get_frames(data, frame_size, hop_size):

  N_FEATURES = 3
  frames = []
  labels = []
  for i in range(0, len(dataCol) - frame_size, hop_size):
    x = data['x'].values[i: i + frame_size]
    y = data['y'].values[i: i + frame_size]
    z = data['z'].values[i: i + frame_size]
        # Retrieve the most often used label in this segment
    label = stats.mode(dataCol['label'][i: i + frame_size])[0][0]
    frames.append([x, y, z])
    labels.append(label)
  # Bring the segments into a better shape
  frames = np.asarray(frames, dtype= np.float32).reshape(-1, frame_size, N_FEATURES)
  labels = np.asarray(labels)
  return frames, labels

from scipy import stats
X,y = get_frames(dataCol, frame_size, hop_size)

X.shape
y.shape

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0, stratify = y)

print(x_train.shape)
print(x_test.shape)

print(y_train.shape)
print(y_test.shape)

import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from keras.layers import Conv1D,MaxPooling1D,Flatten

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, Dropout, MaxPooling1D, Flatten, LSTM,BatchNormalization, SimpleRNN

model = Sequential()
model.add(SimpleRNN(30, input_shape=x_train[0].shape))
model.add((Flatten()))
model.add(Dense(3, activation='softmax'))
model.summary()

from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=0.0001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy',f1_m,precision_m,recall_m])

from time import perf_counter
print("Time Counting Started :")
t1_start = perf_counter()

score = model.fit(x_train,y_train, epochs=30, validation_data= (x_test, y_test),batch_size=64, verbose=1)

t1_stop = perf_counter()
print("Elapsed time:", t1_stop, t1_start)
print("Elapsed time during the whole program in seconds:",
                                        t1_stop-t1_start)

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test)

predy = np.argmax(model.predict(x_test), axis=1)

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report

cm=confusion_matrix(y_test,predy)
print(cm)
print(accuracy_score(y_test,predy))
print("classification_report of model1:")

mat = confusion_matrix(y_test,predy)

print('Precision: %.3f' % precision_score(y_test, predy, average='micro'))
target_names = ['class 0', 'class 1', 'class 2']

print(classification_report(y_test,predy, target_names=target_names))

plot_confusion_matrix(conf_mat=mat, show_normed=True)

plot_confusion_matrix(conf_mat=mat, show_normed=True, show_absolute=False, colorbar=True)

import matplotlib.pyplot as plt1

# Plot training & validation accuracy values
plt.figure(figsize=(4,4))
plt.plot(score.history['accuracy'],label='Train accuracy')
plt.plot(score.history['val_accuracy'],label = 'Validation accuracy')
plt.xlabel('epoch no')
plt.ylabel('acc')
plt.legend()
plt.show()

plt.figure(figsize=(4,4))
plt.plot(score.history['loss'],label='Train loss')
plt.plot(score.history['val_loss'],label = 'Validation loss')
plt.xlabel('epoch no')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
#YCS-2023_WISDM_CNN+GRU_Model_Source_Code
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

file = open('/content/drive/MyDrive/WISDM/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt')
lines = file.readlines()

processedList = []

for i, line in enumerate(lines):
    try:
        line = line.split(',')
        last = line[5].split(';')[0]
        last = last.strip()
        if last == '':
            break;
        temp = [line[0], line[1], line[2], line[3], line[4], last]
        processedList.append(temp)
    except:
        print('Error at line number: ', i)


columns = ['user', 'activity', 'time', 'x', 'y', 'z']

data = pd.DataFrame(data = processedList, columns = columns)
data.head()

data.shape

data.isnull().sum()

data['x'] = data['x'].astype('float')
data['y'] = data['y'].astype('float')
data['z'] = data['z'].astype('float')

activities = data['activity'].value_counts().index
activities

data['activity'].value_counts()

# Normalize features for training data set (values between 0 and 1)
# Surpress warning for next 3 operation
pd.options.mode.chained_assignment = None  # default='warn'
data['x'] = data['x'].astype(float) / data['x'].astype(float).max()
data['y'] = data['y'].astype(float) / data['y'].astype(float).max()
data['z'] = data['z'].astype(float) / data['z'].astype(float).max()
# Round numbers
data = data.round({'x': 4, 'y': 4, 'z': 4})

static = data[data['activity'].isin (['Sitting', 'Standing'])]
static

static['Encode'] = "Static"

static

gradual = data[data['activity'].isin (['Upstairs', 'Downstairs'])]
gradual

gradual['Encode'] = "Gradual"
gradual

dynamic = data[data['activity'].isin (['Walking', 'Jogging'])]
dynamic

dynamic['Encode'] = "Dynamic"
dynamic

frames = [static, gradual, dynamic
          ]
dataCol = pd.concat(frames)
print(dataCol)
dataCol['Encode'].value_counts()

dataCol

from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
dataCol['label'] = label.fit_transform(dataCol['Encode'])
dataCol.head()

dataCol['label'].value_counts()

dataCol = dataCol.drop(['user', 'activity', 'time', 'Encode'], axis = 1).copy()
dataCol.head()

label.classes_

Fs = 50
frame_size = Fs*4 # 200
hop_size = Fs*2 # 200

import math as m
def get_frames(data, frame_size, hop_size):

  N_FEATURES = 3
  frames = []
  labels = []
  for i in range(0, len(dataCol) - frame_size, hop_size):
    x = data['x'].values[i: i + frame_size]
    y = data['y'].values[i: i + frame_size]
    z = data['z'].values[i: i + frame_size]
        # Retrieve the most often used label in this segment
    label = stats.mode(dataCol['label'][i: i + frame_size])[0][0]
    frames.append([x, y, z])
    labels.append(label)
  # Bring the segments into a better shape
  frames = np.asarray(frames, dtype= np.float32).reshape(-1, frame_size, N_FEATURES)
  labels = np.asarray(labels)
  return frames, labels

from scipy import stats
X,y = get_frames(dataCol, frame_size, hop_size)

X.shape
y.shape

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0, stratify = y)

print(x_train.shape)
print(x_test.shape)

print(y_train.shape)
print(y_test.shape)

import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from keras.layers import Conv1D,MaxPooling1D,Flatten

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, Dropout, MaxPooling1D, Flatten, LSTM,BatchNormalization, SimpleRNN

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu',input_shape=x_train[0].shape))
model.add((GRU(30)))
model.add((Flatten()))
model.add(Dense(3, activation='softmax'))
model.summary()

from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=0.0001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy',f1_m,precision_m,recall_m])

from time import perf_counter
print("Time Counting Started :")
t1_start = perf_counter()

score = model.fit(x_train,y_train, epochs=30, validation_data= (x_test, y_test),batch_size=64, verbose=1)

t1_stop = perf_counter()
print("Elapsed time:", t1_stop, t1_start)
print("Elapsed time during the whole program in seconds:",
                                        t1_stop-t1_start)

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test)

predy = np.argmax(model.predict(x_test), axis=1)

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report

cm=confusion_matrix(y_test,predy)
print(cm)
print(accuracy_score(y_test,predy))
print("classification_report of model1:")

mat = confusion_matrix(y_test,predy)

print('Precision: %.3f' % precision_score(y_test, predy, average='micro'))
target_names = ['class 0', 'class 1', 'class 2']

print(classification_report(y_test,predy, target_names=target_names))

plot_confusion_matrix(conf_mat=mat, show_normed=True)

plot_confusion_matrix(conf_mat=mat, show_normed=True, show_absolute=False, colorbar=True)

import matplotlib.pyplot as plt1

# Plot training & validation accuracy values
plt.figure(figsize=(4,4))
plt.plot(score.history['accuracy'],label='Train accuracy')
plt.plot(score.history['val_accuracy'],label = 'Validation accuracy')
plt.xlabel('epoch no')
plt.ylabel('acc')
plt.legend()
plt.show()

plt.figure(figsize=(4,4))
plt.plot(score.history['loss'],label='Train loss')
plt.plot(score.history['val_loss'],label = 'Validation loss')
plt.xlabel('epoch no')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
#YCS-2023_WISDM_CNN+Bi-LSTM_Model_Source_Code
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

file = open('/content/drive/MyDrive/WISDM/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt')
lines = file.readlines()

processedList = []

for i, line in enumerate(lines):
    try:
        line = line.split(',')
        last = line[5].split(';')[0]
        last = last.strip()
        if last == '':
            break;
        temp = [line[0], line[1], line[2], line[3], line[4], last]
        processedList.append(temp)
    except:
        print('Error at line number: ', i)


columns = ['user', 'activity', 'time', 'x', 'y', 'z']

data = pd.DataFrame(data = processedList, columns = columns)
data.head()

data.shape

data.isnull().sum()

data['x'] = data['x'].astype('float')
data['y'] = data['y'].astype('float')
data['z'] = data['z'].astype('float')

activities = data['activity'].value_counts().index
activities

data['activity'].value_counts()

# Normalize features for training data set (values between 0 and 1)
# Surpress warning for next 3 operation
pd.options.mode.chained_assignment = None  # default='warn'
data['x'] = data['x'].astype(float) / data['x'].astype(float).max()
data['y'] = data['y'].astype(float) / data['y'].astype(float).max()
data['z'] = data['z'].astype(float) / data['z'].astype(float).max()
# Round numbers
data = data.round({'x': 4, 'y': 4, 'z': 4})

static = data[data['activity'].isin (['Sitting', 'Standing'])]
static

static['Encode'] = "Static"

static

gradual = data[data['activity'].isin (['Upstairs', 'Downstairs'])]
gradual

gradual['Encode'] = "Gradual"
gradual

dynamic = data[data['activity'].isin (['Walking', 'Jogging'])]
dynamic

dynamic['Encode'] = "Dynamic"
dynamic

frames = [static, gradual, dynamic
          ]
dataCol = pd.concat(frames)
print(dataCol)
dataCol['Encode'].value_counts()

dataCol

from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
dataCol['label'] = label.fit_transform(dataCol['Encode'])
dataCol.head()

dataCol['label'].value_counts()

dataCol = dataCol.drop(['user', 'activity', 'time', 'Encode'], axis = 1).copy()
dataCol.head()

label.classes_

Fs = 50
frame_size = Fs*4 # 200
hop_size = Fs*2 # 200

import math as m
def get_frames(data, frame_size, hop_size):

  N_FEATURES = 3
  frames = []
  labels = []
  for i in range(0, len(dataCol) - frame_size, hop_size):
    x = data['x'].values[i: i + frame_size]
    y = data['y'].values[i: i + frame_size]
    z = data['z'].values[i: i + frame_size]
        # Retrieve the most often used label in this segment
    label = stats.mode(dataCol['label'][i: i + frame_size])[0][0]
    frames.append([x, y, z])
    labels.append(label)
  # Bring the segments into a better shape
  frames = np.asarray(frames, dtype= np.float32).reshape(-1, frame_size, N_FEATURES)
  labels = np.asarray(labels)
  return frames, labels

from scipy import stats
X,y = get_frames(dataCol, frame_size, hop_size)

X.shape
y.shape

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0, stratify = y)

print(x_train.shape)
print(x_test.shape)

print(y_train.shape)
print(y_test.shape)

import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from keras.layers import Conv1D,MaxPooling1D,Flatten

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, Dropout, MaxPooling1D, Flatten, LSTM,BatchNormalization, SimpleRNN

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu',input_shape=x_train[0].shape))
model.add(Bidirectional(LSTM(30)))
model.add((Flatten()))
model.add(Dense(3, activation="softmax"))
model.summary()

from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=0.0001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy',f1_m,precision_m,recall_m])

from time import perf_counter
print("Time Counting Started :")
t1_start = perf_counter()

score = model.fit(x_train,y_train, epochs=30, validation_data= (x_test, y_test),batch_size=64, verbose=1)

t1_stop = perf_counter()
print("Elapsed time:", t1_stop, t1_start)
print("Elapsed time during the whole program in seconds:",
                                        t1_stop-t1_start)

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test)

predy = np.argmax(model.predict(x_test), axis=1)

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report

cm=confusion_matrix(y_test,predy)
print(cm)
print(accuracy_score(y_test,predy))
print("classification_report of model1:")

mat = confusion_matrix(y_test,predy)

print('Precision: %.3f' % precision_score(y_test, predy, average='micro'))
target_names = ['class 0', 'class 1', 'class 2']

print(classification_report(y_test,predy, target_names=target_names))

plot_confusion_matrix(conf_mat=mat, show_normed=True)

plot_confusion_matrix(conf_mat=mat, show_normed=True, show_absolute=False, colorbar=True)

import matplotlib.pyplot as plt1

# Plot training & validation accuracy values
plt.figure(figsize=(4,4))
plt.plot(score.history['accuracy'],label='Train accuracy')
plt.plot(score.history['val_accuracy'],label = 'Validation accuracy')
plt.xlabel('epoch no')
plt.ylabel('acc')
plt.legend()
plt.show()

plt.figure(figsize=(4,4))
plt.plot(score.history['loss'],label='Train loss')
plt.plot(score.history['val_loss'],label = 'Validation loss')
plt.xlabel('epoch no')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
#YCS-2023_WISDM_CNN+RNN_Model_Source_Code
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

file = open('/content/drive/MyDrive/WISDM/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt')
lines = file.readlines()

processedList = []

for i, line in enumerate(lines):
    try:
        line = line.split(',')
        last = line[5].split(';')[0]
        last = last.strip()
        if last == '':
            break;
        temp = [line[0], line[1], line[2], line[3], line[4], last]
        processedList.append(temp)
    except:
        print('Error at line number: ', i)


columns = ['user', 'activity', 'time', 'x', 'y', 'z']

data = pd.DataFrame(data = processedList, columns = columns)
data.head()

data.shape

data.isnull().sum()

data['x'] = data['x'].astype('float')
data['y'] = data['y'].astype('float')
data['z'] = data['z'].astype('float')

activities = data['activity'].value_counts().index
activities

data['activity'].value_counts()

# Normalize features for training data set (values between 0 and 1)
# Surpress warning for next 3 operation
pd.options.mode.chained_assignment = None  # default='warn'
data['x'] = data['x'].astype(float) / data['x'].astype(float).max()
data['y'] = data['y'].astype(float) / data['y'].astype(float).max()
data['z'] = data['z'].astype(float) / data['z'].astype(float).max()
# Round numbers
data = data.round({'x': 4, 'y': 4, 'z': 4})

static = data[data['activity'].isin (['Sitting', 'Standing'])]
static

static['Encode'] = "Static"

static

gradual = data[data['activity'].isin (['Upstairs', 'Downstairs'])]
gradual

gradual['Encode'] = "Gradual"
gradual

dynamic = data[data['activity'].isin (['Walking', 'Jogging'])]
dynamic

dynamic['Encode'] = "Dynamic"
dynamic

frames = [static, gradual, dynamic
          ]
dataCol = pd.concat(frames)
print(dataCol)
dataCol['Encode'].value_counts()

dataCol

from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
dataCol['label'] = label.fit_transform(dataCol['Encode'])
dataCol.head()

dataCol['label'].value_counts()

dataCol = dataCol.drop(['user', 'activity', 'time', 'Encode'], axis = 1).copy()
dataCol.head()

label.classes_

Fs = 50
frame_size = Fs*4 # 200
hop_size = Fs*2 # 200

import math as m
def get_frames(data, frame_size, hop_size):

  N_FEATURES = 3
  frames = []
  labels = []
  for i in range(0, len(dataCol) - frame_size, hop_size):
    x = data['x'].values[i: i + frame_size]
    y = data['y'].values[i: i + frame_size]
    z = data['z'].values[i: i + frame_size]
        # Retrieve the most often used label in this segment
    label = stats.mode(dataCol['label'][i: i + frame_size])[0][0]
    frames.append([x, y, z])
    labels.append(label)
  # Bring the segments into a better shape
  frames = np.asarray(frames, dtype= np.float32).reshape(-1, frame_size, N_FEATURES)
  labels = np.asarray(labels)
  return frames, labels

from scipy import stats
X,y = get_frames(dataCol, frame_size, hop_size)

X.shape
y.shape

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0, stratify = y)

print(x_train.shape)
print(x_test.shape)

print(y_train.shape)
print(y_test.shape)

import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from keras.layers import Conv1D,MaxPooling1D,Flatten

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, Dropout, MaxPooling1D, Flatten, LSTM,BatchNormalization, SimpleRNN

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu',input_shape=x_train[0].shape))
model.add(SimpleRNN(32))
model.add(Activation("relu"))
model.add((Flatten()))
model.add(Dense(3, activation="softmax"))
model.summary()

from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=0.0001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy',f1_m,precision_m,recall_m])

from time import perf_counter
print("Time Counting Started :")
t1_start = perf_counter()

score = model.fit(x_train,y_train, epochs=30, validation_data= (x_test, y_test),batch_size=64, verbose=1)

t1_stop = perf_counter()
print("Elapsed time:", t1_stop, t1_start)
print("Elapsed time during the whole program in seconds:",
                                        t1_stop-t1_start)

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test)

predy = np.argmax(model.predict(x_test), axis=1)

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report

cm=confusion_matrix(y_test,predy)
print(cm)
print(accuracy_score(y_test,predy))
print("classification_report of model1:")

mat = confusion_matrix(y_test,predy)

print('Precision: %.3f' % precision_score(y_test, predy, average='micro'))
target_names = ['class 0', 'class 1', 'class 2']

print(classification_report(y_test,predy, target_names=target_names))

plot_confusion_matrix(conf_mat=mat, show_normed=True)

plot_confusion_matrix(conf_mat=mat, show_normed=True, show_absolute=False, colorbar=True)

import matplotlib.pyplot as plt1

# Plot training & validation accuracy values
plt.figure(figsize=(4,4))
plt.plot(score.history['accuracy'],label='Train accuracy')
plt.plot(score.history['val_accuracy'],label = 'Validation accuracy')
plt.xlabel('epoch no')
plt.ylabel('acc')
plt.legend()
plt.show()

plt.figure(figsize=(4,4))
plt.plot(score.history['loss'],label='Train loss')
plt.plot(score.history['val_loss'],label = 'Validation loss')
plt.xlabel('epoch no')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
#YCS-2023_WISDM_GRU_Model_Source_Code
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

file = open('/content/drive/MyDrive/WISDM/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt')
lines = file.readlines()

processedList = []

for i, line in enumerate(lines):
    try:
        line = line.split(',')
        last = line[5].split(';')[0]
        last = last.strip()
        if last == '':
            break;
        temp = [line[0], line[1], line[2], line[3], line[4], last]
        processedList.append(temp)
    except:
        print('Error at line number: ', i)


columns = ['user', 'activity', 'time', 'x', 'y', 'z']

data = pd.DataFrame(data = processedList, columns = columns)
data.head()

data.shape

data.isnull().sum()

data['x'] = data['x'].astype('float')
data['y'] = data['y'].astype('float')
data['z'] = data['z'].astype('float')

activities = data['activity'].value_counts().index
activities

data['activity'].value_counts()

# Normalize features for training data set (values between 0 and 1)
# Surpress warning for next 3 operation
pd.options.mode.chained_assignment = None  # default='warn'
data['x'] = data['x'].astype(float) / data['x'].astype(float).max()
data['y'] = data['y'].astype(float) / data['y'].astype(float).max()
data['z'] = data['z'].astype(float) / data['z'].astype(float).max()
# Round numbers
data = data.round({'x': 4, 'y': 4, 'z': 4})

static = data[data['activity'].isin (['Sitting', 'Standing'])]
static

static['Encode'] = "Static"

static

gradual = data[data['activity'].isin (['Upstairs', 'Downstairs'])]
gradual

gradual['Encode'] = "Gradual"
gradual

dynamic = data[data['activity'].isin (['Walking', 'Jogging'])]
dynamic

dynamic['Encode'] = "Dynamic"
dynamic

frames = [static, gradual, dynamic
          ]
dataCol = pd.concat(frames)
print(dataCol)
dataCol['Encode'].value_counts()

dataCol

from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
dataCol['label'] = label.fit_transform(dataCol['Encode'])
dataCol.head()

dataCol['label'].value_counts()

dataCol = dataCol.drop(['user', 'activity', 'time', 'Encode'], axis = 1).copy()
dataCol.head()

label.classes_

Fs = 50
frame_size = Fs*4 # 200
hop_size = Fs*2 # 200

import math as m
def get_frames(data, frame_size, hop_size):

  N_FEATURES = 3
  frames = []
  labels = []
  for i in range(0, len(dataCol) - frame_size, hop_size):
    x = data['x'].values[i: i + frame_size]
    y = data['y'].values[i: i + frame_size]
    z = data['z'].values[i: i + frame_size]
        # Retrieve the most often used label in this segment
    label = stats.mode(dataCol['label'][i: i + frame_size])[0][0]
    frames.append([x, y, z])
    labels.append(label)
  # Bring the segments into a better shape
  frames = np.asarray(frames, dtype= np.float32).reshape(-1, frame_size, N_FEATURES)
  labels = np.asarray(labels)
  return frames, labels

from scipy import stats
X,y = get_frames(dataCol, frame_size, hop_size)

X.shape
y.shape

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0, stratify = y)

print(x_train.shape)
print(x_test.shape)

print(y_train.shape)
print(y_test.shape)

import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from keras.layers import Conv1D,MaxPooling1D,Flatten

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, Dropout, MaxPooling1D, Flatten, LSTM,BatchNormalization, SimpleRNN

model = Sequential()
model.add(GRU(30, return_sequences=True, input_shape=x_train[0].shape))
model.add((GRU(30)))
model.add((Flatten()))
model.add(Dense(3, activation='softmax'))
model.summary()

from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=0.0001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy',f1_m,precision_m,recall_m])

from time import perf_counter
print("Time Counting Started :")
t1_start = perf_counter()

score = model.fit(x_train,y_train, epochs=30, validation_data= (x_test, y_test),batch_size=64, verbose=1)

t1_stop = perf_counter()
print("Elapsed time:", t1_stop, t1_start)
print("Elapsed time during the whole program in seconds:",
                                        t1_stop-t1_start)

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test)

predy = np.argmax(model.predict(x_test), axis=1)

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report

cm=confusion_matrix(y_test,predy)
print(cm)
print(accuracy_score(y_test,predy))
print("classification_report of model1:")

mat = confusion_matrix(y_test,predy)

print('Precision: %.3f' % precision_score(y_test, predy, average='micro'))
target_names = ['class 0', 'class 1', 'class 2']

print(classification_report(y_test,predy, target_names=target_names))

plot_confusion_matrix(conf_mat=mat, show_normed=True)

plot_confusion_matrix(conf_mat=mat, show_normed=True, show_absolute=False, colorbar=True)

import matplotlib.pyplot as plt1

# Plot training & validation accuracy values
plt.figure(figsize=(4,4))
plt.plot(score.history['accuracy'],label='Train accuracy')
plt.plot(score.history['val_accuracy'],label = 'Validation accuracy')
plt.xlabel('epoch no')
plt.ylabel('acc')
plt.legend()
plt.show()

plt.figure(figsize=(4,4))
plt.plot(score.history['loss'],label='Train loss')
plt.plot(score.history['val_loss'],label = 'Validation loss')
plt.xlabel('epoch no')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
#YCS-2023_WISDM_Bi-LSTM_Model_Source_Code
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

file = open('/content/drive/MyDrive/WISDM/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt')
lines = file.readlines()

processedList = []

for i, line in enumerate(lines):
    try:
        line = line.split(',')
        last = line[5].split(';')[0]
        last = last.strip()
        if last == '':
            break;
        temp = [line[0], line[1], line[2], line[3], line[4], last]
        processedList.append(temp)
    except:
        print('Error at line number: ', i)


columns = ['user', 'activity', 'time', 'x', 'y', 'z']

data = pd.DataFrame(data = processedList, columns = columns)
data.head()

data.shape

data.isnull().sum()

data['x'] = data['x'].astype('float')
data['y'] = data['y'].astype('float')
data['z'] = data['z'].astype('float')

activities = data['activity'].value_counts().index
activities

data['activity'].value_counts()

# Normalize features for training data set (values between 0 and 1)
# Surpress warning for next 3 operation
pd.options.mode.chained_assignment = None  # default='warn'
data['x'] = data['x'].astype(float) / data['x'].astype(float).max()
data['y'] = data['y'].astype(float) / data['y'].astype(float).max()
data['z'] = data['z'].astype(float) / data['z'].astype(float).max()
# Round numbers
data = data.round({'x': 4, 'y': 4, 'z': 4})

static = data[data['activity'].isin (['Sitting', 'Standing'])]
static

static['Encode'] = "Static"

static

gradual = data[data['activity'].isin (['Upstairs', 'Downstairs'])]
gradual

gradual['Encode'] = "Gradual"
gradual

dynamic = data[data['activity'].isin (['Walking', 'Jogging'])]
dynamic

dynamic['Encode'] = "Dynamic"
dynamic

frames = [static, gradual, dynamic
          ]
dataCol = pd.concat(frames)
print(dataCol)
dataCol['Encode'].value_counts()

dataCol

from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
dataCol['label'] = label.fit_transform(dataCol['Encode'])
dataCol.head()

dataCol['label'].value_counts()

dataCol = dataCol.drop(['user', 'activity', 'time', 'Encode'], axis = 1).copy()
dataCol.head()

label.classes_

Fs = 50
frame_size = Fs*4 # 200
hop_size = Fs*2 # 200

import math as m
def get_frames(data, frame_size, hop_size):

  N_FEATURES = 3
  frames = []
  labels = []
  for i in range(0, len(dataCol) - frame_size, hop_size):
    x = data['x'].values[i: i + frame_size]
    y = data['y'].values[i: i + frame_size]
    z = data['z'].values[i: i + frame_size]
        # Retrieve the most often used label in this segment
    label = stats.mode(dataCol['label'][i: i + frame_size])[0][0]
    frames.append([x, y, z])
    labels.append(label)
  # Bring the segments into a better shape
  frames = np.asarray(frames, dtype= np.float32).reshape(-1, frame_size, N_FEATURES)
  labels = np.asarray(labels)
  return frames, labels

from scipy import stats
X,y = get_frames(dataCol, frame_size, hop_size)

X.shape
y.shape

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0, stratify = y)

print(x_train.shape)
print(x_test.shape)

print(y_train.shape)
print(y_test.shape)

import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from keras.layers import Conv1D,MaxPooling1D,Flatten

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, Dropout, MaxPooling1D, Flatten, LSTM,BatchNormalization, SimpleRNN

model = Sequential()
model.add(Bidirectional(LSTM(30, return_sequences=True), input_shape=x_train[0].shape))
model.add(Bidirectional(LSTM(30)))
model.add((Flatten()))
model.add(Dense(3, activation='softmax'))
model.summary()

from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=0.0001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy',f1_m,precision_m,recall_m])

from time import perf_counter
print("Time Counting Started :")
t1_start = perf_counter()

score = model.fit(x_train,y_train, epochs=30, validation_data= (x_test, y_test),batch_size=64, verbose=1)

t1_stop = perf_counter()
print("Elapsed time:", t1_stop, t1_start)
print("Elapsed time during the whole program in seconds:",
                                        t1_stop-t1_start)

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test)

predy = np.argmax(model.predict(x_test), axis=1)

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report

cm=confusion_matrix(y_test,predy)
print(cm)
print(accuracy_score(y_test,predy))
print("classification_report of model1:")

mat = confusion_matrix(y_test,predy)

print('Precision: %.3f' % precision_score(y_test, predy, average='micro'))
target_names = ['class 0', 'class 1', 'class 2']

print(classification_report(y_test,predy, target_names=target_names))

plot_confusion_matrix(conf_mat=mat, show_normed=True)

plot_confusion_matrix(conf_mat=mat, show_normed=True, show_absolute=False, colorbar=True)

import matplotlib.pyplot as plt1

# Plot training & validation accuracy values
plt.figure(figsize=(4,4))
plt.plot(score.history['accuracy'],label='Train accuracy')
plt.plot(score.history['val_accuracy'],label = 'Validation accuracy')
plt.xlabel('epoch no')
plt.ylabel('acc')
plt.legend()
plt.show()

plt.figure(figsize=(4,4))
plt.plot(score.history['loss'],label='Train loss')
plt.plot(score.history['val_loss'],label = 'Validation loss')
plt.xlabel('epoch no')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
#YCS-2023_WISDM_LSTM_Model_Source_Code
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

file = open('/content/drive/MyDrive/WISDM/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt')
lines = file.readlines()

processedList = []

for i, line in enumerate(lines):
    try:
        line = line.split(',')
        last = line[5].split(';')[0]
        last = last.strip()
        if last == '':
            break;
        temp = [line[0], line[1], line[2], line[3], line[4], last]
        processedList.append(temp)
    except:
        print('Error at line number: ', i)


columns = ['user', 'activity', 'time', 'x', 'y', 'z']

data = pd.DataFrame(data = processedList, columns = columns)
data.head()

data.shape

data.isnull().sum()

data['x'] = data['x'].astype('float')
data['y'] = data['y'].astype('float')
data['z'] = data['z'].astype('float')

activities = data['activity'].value_counts().index
activities

data['activity'].value_counts()

# Normalize features for training data set (values between 0 and 1)
# Surpress warning for next 3 operation
pd.options.mode.chained_assignment = None  # default='warn'
data['x'] = data['x'].astype(float) / data['x'].astype(float).max()
data['y'] = data['y'].astype(float) / data['y'].astype(float).max()
data['z'] = data['z'].astype(float) / data['z'].astype(float).max()
# Round numbers
data = data.round({'x': 4, 'y': 4, 'z': 4})

static = data[data['activity'].isin (['Sitting', 'Standing'])]
static

static['Encode'] = "Static"

static

gradual = data[data['activity'].isin (['Upstairs', 'Downstairs'])]
gradual

gradual['Encode'] = "Gradual"
gradual

dynamic = data[data['activity'].isin (['Walking', 'Jogging'])]
dynamic

dynamic['Encode'] = "Dynamic"
dynamic

frames = [static, gradual, dynamic
          ]
dataCol = pd.concat(frames)
print(dataCol)
dataCol['Encode'].value_counts()

dataCol

from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
dataCol['label'] = label.fit_transform(dataCol['Encode'])
dataCol.head()

dataCol['label'].value_counts()

dataCol = dataCol.drop(['user', 'activity', 'time', 'Encode'], axis = 1).copy()
dataCol.head()

label.classes_

Fs = 50
frame_size = Fs*4 # 200
hop_size = Fs*2 # 200

import math as m
def get_frames(data, frame_size, hop_size):

  N_FEATURES = 3
  frames = []
  labels = []
  for i in range(0, len(dataCol) - frame_size, hop_size):
    x = data['x'].values[i: i + frame_size]
    y = data['y'].values[i: i + frame_size]
    z = data['z'].values[i: i + frame_size]
        # Retrieve the most often used label in this segment
    label = stats.mode(dataCol['label'][i: i + frame_size])[0][0]
    frames.append([x, y, z])
    labels.append(label)
  # Bring the segments into a better shape
  frames = np.asarray(frames, dtype= np.float32).reshape(-1, frame_size, N_FEATURES)
  labels = np.asarray(labels)
  return frames, labels

from scipy import stats
X,y = get_frames(dataCol, frame_size, hop_size)

X.shape
y.shape

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0, stratify = y)

print(x_train.shape)
print(x_test.shape)

print(y_train.shape)
print(y_test.shape)

import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from keras.layers import Conv1D,MaxPooling1D,Flatten

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, Dropout, MaxPooling1D, Flatten, LSTM,BatchNormalization, SimpleRNN

model = Sequential()
model.add(LSTM(30, return_sequences=True, input_shape=x_train[0].shape))
model.add((LSTM(30)))
model.add((Flatten()))
model.add(Dense(3, activation='softmax'))
model.summary()

from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=0.0001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy',f1_m,precision_m,recall_m])

from time import perf_counter
print("Time Counting Started :")
t1_start = perf_counter()

score = model.fit(x_train,y_train, epochs=30, validation_data= (x_test, y_test),batch_size=64, verbose=1)

t1_stop = perf_counter()
print("Elapsed time:", t1_stop, t1_start)
print("Elapsed time during the whole program in seconds:",
                                        t1_stop-t1_start)

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test)

predy = np.argmax(model.predict(x_test), axis=1)

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report

cm=confusion_matrix(y_test,predy)
print(cm)
print(accuracy_score(y_test,predy))
print("classification_report of model1:")

mat = confusion_matrix(y_test,predy)

print('Precision: %.3f' % precision_score(y_test, predy, average='micro'))
target_names = ['class 0', 'class 1', 'class 2']

print(classification_report(y_test,predy, target_names=target_names))

plot_confusion_matrix(conf_mat=mat, show_normed=True)

plot_confusion_matrix(conf_mat=mat, show_normed=True, show_absolute=False, colorbar=True)

import matplotlib.pyplot as plt1

# Plot training & validation accuracy values
plt.figure(figsize=(4,4))
plt.plot(score.history['accuracy'],label='Train accuracy')
plt.plot(score.history['val_accuracy'],label = 'Validation accuracy')
plt.xlabel('epoch no')
plt.ylabel('acc')
plt.legend()
plt.show()

plt.figure(figsize=(4,4))
plt.plot(score.history['loss'],label='Train loss')
plt.plot(score.history['val_loss'],label = 'Validation loss')
plt.xlabel('epoch no')
plt.ylabel('loss')
plt.legend()
plt.show()